# Исследование надежности заемщиков


## Откроем таблицу и изучим общую информацию о данных

Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`. Путь к файлу:

`/datasets/data.csv`

Выведем первые 20 строчек датафрейма `data` на экран.

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. 

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Обработаем значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.

In [6]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть.

Выведем перечень уникальных значений столбца `children`.

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру.

In [14]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
child_debt = data.groupby('children')['debt'].sum()/data.groupby('children')['debt'].count()
print(child_debt) # Здесь отдельно вывожу процентную долю должников относительно количеству детей
group_child_debt = data.groupby('children').agg({'debt': ['count', 'sum', 'mean']})
group_child_debt # Получаем таблицу с тремя метриками: count-общее количество заёмщиков,
# sum-количество должников в выбранной категории, mean-среднее значение должников в процентном соотношении

children
0    0.075438
1    0.092346
2    0.094542
3    0.081818
4    0.097561
5    0.000000
Name: debt, dtype: float64


debt                
          count   sum      mean
children                       
0         14091  1063  0.075438
1          4808   444  0.092346
2          2052   194  0.094542
3           330    27  0.081818
4            41     4  0.097561
5             9     0  0.000000

**Вывод:** Количество детей влияет на возврат кредита в срок. Так как категория клиентов без детей составляет основную аудиторию заёмщиков, и при этом имеет самый низкий процент должников - можно сделать вывод что эта категория наиболее надежный заёмщик возвращающий кредит в срок.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
group_status_debt = data.groupby('family_status').agg({'debt': ['count', 'sum', 'mean']})
group_status_debt # так же, count-общее число заёмщиков, sum-должники, mean-процентное соотношение

debt               
                       count  sum      mean
family_status                              
Не женат / не замужем   2796  273  0.097639
в разводе               1189   84  0.070648
вдовец / вдова           951   63  0.066246
гражданский брак        4134  385  0.093130
женат / замужем        12261  927  0.075606

**Вывод:** Люди не повязанные узами брака, а так же те кто в гражданском браке возвращают кредит реже. Возможно это связано с тем, что им нечего терять в случае негативного исхода событий. По остальным группам разница незначительна, и не превышает процентное соотношение вышеуказанных. Однако, в строке женат/замужем можно увидеть увеличение общего количества заёмщиков, и количество должников закономерно больше чем в остальных строках. А значит, вероятнее получить недобросовестного клиента со статусом женат/замужем выше, нежели не замужних, например. В силу того что в последней строке существенно большее количество клиентов. 

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
group_income_debt = data.groupby('total_income_category').agg({'debt': ['count', 'sum', 'mean']}) # сгруппировал по функции 
# рассчитанной раннее, где мы указывали категории заработка заёмщиков. Е-до 30к, D-от 30 до 50к, С-от 50 до 200к, В-от 200 до 1кк, А-свыше 1кк (чтоб я так жил)
group_income_debt

debt                
                       count   sum      mean
total_income_category                       
A                         25     2  0.080000
B                       5014   354  0.070602
C                      15921  1353  0.084982
D                        349    21  0.060172
E                         22     2  0.090909

**Вывод:** Здесь можно увидеть большой численный перевес заёмщиков у которых заработок соответствует категории С. Не смотря на несущественную разницу между всеми категориями в процентном соотношении, в категории С все же больше вероятности того, что заёмщик просрочит выплату, просто потому что это основная аудитория заёмщиков.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

In [25]:
# Ваш код будет здесь. Вы можете создавать новые ячейки.
group_purpose_debt = data.groupby('purpose_category').agg({'debt': ['count', 'sum', 'mean']})
group_purpose_debt
# как и в предыдущем, использовалась функия при группировке, в которой мы раннее разделили все цели по основным категориям для
# упрощения визуализации

debt               
                          count  sum      mean
purpose_category                              
операции с автомобилем     4279  400  0.093480
операции с недвижимостью  10751  780  0.072551
получение образования      3988  369  0.092528
проведение свадьбы         2313  183  0.079118

**Вывод:** Клиенты с покупкой недвижимости показывают себя наиболее надежными заёмщиками, нежели другие категории. На это указывает наименьший процент просрочки кредита, и превышающая остальные категории общая численность заёмщиков.

### Шаг 4: общий вывод.


 Проведя исследование, мы выявили зависимость между количеством детей, семейным положением, уровнем дохода, целями кредита и возвратом кредита в установленный срок.
Для этого была проделана работа:

1. Выявление, и удаление пропусков в данных
2. Были обработаны аномальные значения для корректного отображения данных
3. Обработаны дубликаты, для точности проводимого исследования
4. Сделана категоризация данных для упрощенного понимания таблиц
5. Проведен анализ данных отвечающий на поставленные задачи

Вывод: 
 Количество детей влияет на возврат кредита в срок. Так как категория клиентов без детей составляет основную аудиторию заёмщиков, и при этом имеет самый низкий процент должников (0.075%) - можно сделать вывод что эта категория наиболее надежный заёмщик возвращающий кредит в срок. 
 
 Люди не повязанные узами брака, а так же те кто в гражданском браке возвращают кредит реже (0.097%). Возможно это связано с тем, что им нечего терять в случае негативного исхода событий. По остальным группам разница незначительна, и не превышает процентное соотношение вышеуказанных. Однако, в строке женат/замужем можно увидеть увеличение общего количества заёмщиков, и количество должников закономерно больше чем в остальных строках. А значит, вероятнее получить недобросовестного клиента со статусом женат/замужем выше, нежели не замужних (0.075%). В силу того что в последней строке существенно большее количество клиентов. 
 
 Заёмщиков, у которых заработок соответствует категории С больше остальных групп. Не смотря на несущественную разницу между всеми категориями в процентном соотношении, в категории С все же больше вероятности того, что заёмщик просрочит выплату, просто потому что это основная аудитория заёмщиков (0.084%).
 
 Клиенты с покупкой недвижимости показывают себя наиболее надежными заёмщиками, нежели другие категории. На это указывает наименьший процент просрочки кредита, и превышающая остальные категории общая численность заёмщиков(0.072%).